# Chapter 4: Deep Neural Networks for Supervised Learning: Classification

---

## Getting Started

Download the data from kaggle https://www.kaggle.com/c/predicting-red-hat-business-value.

In [ ]:
# essentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 67 Sep  6 20:33 kaggle.json


In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c predicting-red-hat-business-value

people.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
act_test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
act_train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
import zipfile
with zipfile.ZipFile("/content/act_train.csv.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")

In [ ]:
import zipfile
with zipfile.ZipFile("/content/people.csv.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")

## Exploring the Data

In [ ]:
# import the datasets 
df = pd.read_csv('act_train.csv')
people = pd.read_csv('people.csv')

In [9]:
# explore the shape of the datasets
print("Shape of df: ", df.shape)
print("Shape of people df: ", people.shape)

Shape of df:  (2197291, 15)
Shape of people df:  (189118, 41)


In [10]:
df.head()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
0,ppl_100,act2_1734928,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 76,0
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
2,ppl_100,act2_3404049,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
3,ppl_100,act2_3651215,2023-08-04,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
4,ppl_100,act2_4109017,2023-08-26,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2197291 entries, 0 to 2197290
Data columns (total 15 columns):
people_id            object
activity_id          object
date                 object
activity_category    object
char_1               object
char_2               object
char_3               object
char_4               object
char_5               object
char_6               object
char_7               object
char_8               object
char_9               object
char_10              object
outcome              int64
dtypes: int64(1), object(14)
memory usage: 251.5+ MB


In [12]:
people.head()

,people_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,char_11,char_12,char_13,char_14,char_15,char_16,char_17,char_18,char_19,char_20,char_21,char_22,char_23,char_24,char_25,char_26,char_27,char_28,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,type 2,type 2,True,False,False,True,True,False,True,False,False,False,False,True,False,False,False,False,False,True,True,False,True,True,False,False,True,True,True,False,36
1,ppl_100002,type 2,group 8688,type 3,2021-01-06,type 28,type 9,type 5,type 3,type 11,type 2,type 4,False,False,True,True,False,False,False,True,False,False,False,False,False,True,False,True,True,True,False,False,True,True,True,True,True,True,True,False,76
2,ppl_100003,type 2,group 33592,type 3,2022-06-10,type 4,type 8,type 5,type 2,type 5,type 2,type 2,True,True,True,True,True,True,False,True,False,True,False,True,True,True,True,True,True,True,True,False,False,True,True,True,True,False,True,True,99
3,ppl_100004,type 2,group 22593,type 3,2022-07-20,type 40,type 25,type 9,type 4,type 16,type 2,type 2,True,True,True,True,True,False,True,True,True,True,True,True,True,True,False,True,True,True,True,True,True,True,True,True,True,True,True,True,76
4,ppl_100006,type 2,group 6534,type 3,2022-07-27,type 40,type 25,type 9,type 3,type 8,type 2,type 2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,True,False,84


In [13]:
people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189118 entries, 0 to 189117
Data columns (total 41 columns):
people_id    189118 non-null object
char_1       189118 non-null object
group_1      189118 non-null object
char_2       189118 non-null object
date         189118 non-null object
char_3       189118 non-null object
char_4       189118 non-null object
char_5       189118 non-null object
char_6       189118 non-null object
char_7       189118 non-null object
char_8       189118 non-null object
char_9       189118 non-null object
char_10      189118 non-null bool
char_11      189118 non-null bool
char_12      189118 non-null bool
char_13      189118 non-null bool
char_14      189118 non-null bool
char_15      189118 non-null bool
char_16      189118 non-null bool
char_17      189118 non-null bool
char_18      189118 non-null bool
char_19      189118 non-null bool
char_20      189118 non-null bool
char_21      189118 non-null bool
char_22      189118 non-null bool
char_23      18

In [14]:
#Calculating the % of Null values in each column for activity data
df.isnull().sum()/df.shape[0]

people_id            0.000000
activity_id          0.000000
date                 0.000000
activity_category    0.000000
char_1               0.928268
char_2               0.928268
char_3               0.928268
char_4               0.928268
char_5               0.928268
char_6               0.928268
char_7               0.928268
char_8               0.928268
char_9               0.928268
char_10              0.071732
outcome              0.000000
dtype: float64

In [15]:
#Calculate the % of null values in for the entire dataset
people.isnull().sum().sum()

0

In [16]:
#Create the list of columns to drop from activity data
columns_to_remove = ["char_"+str(x) for x in np.arange(1,10)]
print("Columns to remove:",columns_to_remove)

#Remove the columns from the activity data
df = df[list(set(df.columns) - set(columns_to_remove))]

#Rename the 2 columns to avoid name clashes in merged data
df = df.rename(columns={"date":"activity_date","char_10":"activity_type"})

#Replace nulls in the activity_type column with the mode
df["activity_type"] = df["activity_type"].fillna(df["activity_type"].mode()[0])

#Print the shape of the final activity dataset
print("Shape of DF:",df.shape)

Columns to remove: ['char_1', 'char_2', 'char_3', 'char_4', 'char_5', 'char_6', 'char_7', 'char_8', 'char_9']
Shape of DF: (2197291, 6)


In [17]:
#Merge the 2 datasets on 'people_id' key
df_new = df.merge(people,on=["people_id"],how="inner")
print("Shape before merging:",df.shape)
print("Shape after merging :",df_new.shape)

Shape before merging: (2197291, 6)
Shape after merging : (2197291, 46)


In [18]:
df_new.head()

,outcome,activity_date,activity_category,activity_type,people_id,activity_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,char_11,char_12,char_13,char_14,char_15,char_16,char_17,char_18,char_19,char_20,char_21,char_22,char_23,char_24,char_25,char_26,char_27,char_28,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,0,2023-08-26,type 4,type 76,ppl_100,act2_1734928,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,type 2,type 2,True,False,False,True,True,False,True,False,False,False,False,True,False,False,False,False,False,True,True,False,True,True,False,False,True,True,True,False,36
1,0,2022-09-27,type 2,type 1,ppl_100,act2_2434093,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,type 2,type 2,True,False,False,True,True,False,True,False,False,False,False,True,False,False,False,False,False,True,True,False,True,True,False,False,True,True,True,False,36
2,0,2022-09-27,type 2,type 1,ppl_100,act2_3404049,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,type 2,type 2,True,False,False,True,True,False,True,False,False,False,False,True,False,False,False,False,False,True,True,False,True,True,False,False,True,True,True,False,36
3,0,2023-08-04,type 2,type 1,ppl_100,act2_3651215,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,type 2,type 2,True,False,False,True,True,False,True,False,False,False,False,True,False,False,False,False,False,True,True,False,True,True,False,False,True,True,True,False,36
4,0,2023-08-26,type 2,type 1,ppl_100,act2_4109017,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,type 2,type 2,True,False,False,True,True,False,True,False,False,False,False,True,False,False,False,False,False,True,True,False,True,True,False,False,True,True,True,False,36


In [19]:
print("Unique values for outcome: ", df_new['outcome'].unique())
print("\nPercentage of distribution for outcome:")
print(df_new["outcome"].value_counts()/df_new.shape[0])

Unique values for outcome:  [0 1]

Percentage of distribution for outcome:
0    0.556046
1    0.443954
Name: outcome, dtype: float64


---

## Data Engineering

In [20]:
#Checking the distinct datatypes in the dataset
print("Distinct DataTypes:",list(df_new.dtypes.unique()))

Distinct DataTypes: [dtype('int64'), dtype('O'), dtype('bool')]


In [21]:
#Create a temp dataset with the datatype of columns
temp = pd.DataFrame(df_new.dtypes)
temp.columns = ["DataType"]

#Create a list with names of all Boolean columns
boolean_columns = temp.index[temp["DataType"] == 'bool'].values
print("Boolean columns - \n",boolean_columns)

#Convert all boolean columns to Binary numeric values
for column in boolean_columns:
  df_new[column] = np.where(df_new[column] == True,1,0)
  print("\nDistinct DataTypes after processing:",df.dtypes.unique())

Boolean columns - 
 ['char_10' 'char_11' 'char_12' 'char_13' 'char_14' 'char_15' 'char_16'
 'char_17' 'char_18' 'char_19' 'char_20' 'char_21' 'char_22' 'char_23'
 'char_24' 'char_25' 'char_26' 'char_27' 'char_28' 'char_29' 'char_30'
 'char_31' 'char_32' 'char_33' 'char_34' 'char_35' 'char_36' 'char_37']

Distinct DataTypes after processing: [dtype('int64') dtype('O')]

Distinct DataTypes after processing: [dtype('int64') dtype('O')]

Distinct DataTypes after processing: [dtype('int64') dtype('O')]

Distinct DataTypes after processing: [dtype('int64') dtype('O')]

Distinct DataTypes after processing: [dtype('int64') dtype('O')]

Distinct DataTypes after processing: [dtype('int64') dtype('O')]

Distinct DataTypes after processing: [dtype('int64') dtype('O')]

Distinct DataTypes after processing: [dtype('int64') dtype('O')]

Distinct DataTypes after processing: [dtype('int64') dtype('O')]

Distinct DataTypes after processing: [dtype('int64') dtype('O')]

Distinct DataTypes after processin

In [22]:
#Extracting the object columns from the above dataframe
categorical_columns = temp.index[temp["DataType"] == 'O'].values

#Check the number of distinct values in each categorical column
for column in categorical_columns:
  print(column+" column has :",str(len(df_new[column].unique()))+" distinct values")

activity_date column has : 411 distinct values
activity_category column has : 7 distinct values
activity_type column has : 6515 distinct values
people_id column has : 151295 distinct values
activity_id column has : 2197291 distinct values
char_1 column has : 2 distinct values
group_1 column has : 29899 distinct values
char_2 column has : 3 distinct values
date column has : 1196 distinct values
char_3 column has : 43 distinct values
char_4 column has : 25 distinct values
char_5 column has : 9 distinct values
char_6 column has : 7 distinct values
char_7 column has : 25 distinct values
char_8 column has : 8 distinct values
char_9 column has : 9 distinct values


In [23]:
#Create date related features for 'date' in customer data
df_new["date"] = pd.to_datetime(df_new["date"])
df_new["Year"] = df_new["date"].dt.year
df_new["Month"] = df_new["date"].dt.month
df_new["Quarter"] = df_new["date"].dt.quarter
df_new["Week"] = df_new["date"].dt.week
df_new["WeekDay"] = df_new["date"].dt.weekday
df_new["Day"] = df_new["date"].dt.day

#Create date related features for 'date' in activity data
df_new["activity_date"] = pd.to_datetime(df_new["activity_date"])
df_new["Activity_Year"] = df_new["activity_date"].dt.year
df_new["Activity_Month"] = df_new["activity_date"].dt.month
df_new["Activity_Quarter"] = df_new["activity_date"].dt.quarter
df_new["Activity_Week"] = df_new["activity_date"].dt.week
df_new["Activity_WeekDay"] = df_new["activity_date"].dt.weekday
df_new["Activity_Day"] = df_new["activity_date"].dt.day

#Delete the original date columns
del(df_new["date"])
del(df_new["activity_date"])
print("Shape of data after create Date Features:",df_new.shape)

Shape of data after create Date Features: (2197291, 56)


In [24]:
# the remaining categorical columns, which have very high numbers of distinct values
print(df_new[["people_id","activity_type","activity_id",
"group_1"]].head())

  people_id activity_type   activity_id      group_1
0   ppl_100       type 76  act2_1734928  group 17304
1   ppl_100        type 1  act2_2434093  group 17304
2   ppl_100        type 1  act2_3404049  group 17304
3   ppl_100        type 1  act2_3651215  group 17304
4   ppl_100        type 1  act2_4109017  group 17304


In [25]:
#For people ID, we would need to extract values after '_'
df_new.people_id = df_new.people_id.apply(lambda x:x.split("_")[1])
df_new.people_id = pd.to_numeric(df_new.people_id)

#For activity ID also, we would need to extract values after '_'
df_new.activity_id = df_new.activity_id.apply(lambda x:x.split("_")[1])
df_new.activity_id = pd.to_numeric(df_new.activity_id)

#For group_1 , we would need to extract values after "
df_new.group_1 = df_new.group_1.apply(lambda x:x.split(' ')[1])
df_new.group_1 = pd.to_numeric(df_new.group_1)

#For activity_type , we would need to extract values after "
df_new.activity_type = df_new.activity_type.apply(lambda x:x.split(' ')[1])
df_new.activity_type = pd.to_numeric(df_new.activity_type)

#Double check the new values in the dataframe
print(df_new[["people_id","activity_type","activity_id",
              "group_1"]].head())

   people_id  activity_type  activity_id  group_1
0      100.0             76    1734928.0    17304
1      100.0              1    2434093.0    17304
2      100.0              1    3404049.0    17304
3      100.0              1    3651215.0    17304
4      100.0              1    4109017.0    17304


In [31]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Define a function that will intake the raw dataframe and 
# the column name and return a one hot encoded DF
def create_ohe(df, col):
  le = LabelEncoder()
  a = le.fit_transform(df_new[col]).reshape(-1,1)
  ohe = OneHotEncoder(sparse=False)
  column_names = [col+ "_"+ str(i) for i in le.classes_]
  return(pd.DataFrame(ohe.fit_transform(a),columns=column_names))

#Since the above function converts the column, one at a time
#We create a loop to create the final dataset with all features
target = ["outcome"]
numeric_columns = list(set(temp.index[(temp.dtypes =="float64") |
                      (temp.dtypes =="int64")].values) - set(target))
temp = df_new[numeric_columns]
for column in categorical_columns:
  temp_df = create_ohe(df_new,column)
  temp = pd.concat([temp,temp_df],axis=1)

print("\nShape of final df after onehot encoding:",temp.shape)

IndexError: ignored